# Week 2: Vehicle Dynamics & Kinematics

## Module I: Introduction & Foundations

### Topics Covered

- Vehicle Modeling (Kinematic Bicycle Model, Dynamic Bicycle Model)
- Coordinate Frames and Transformations
- Ackermann Steering Geometry

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Derive and implement the kinematic bicycle model for vehicle motion
2. Understand the dynamic bicycle model with tire forces
3. Convert between different coordinate frames (global, vehicle, sensor)
4. Explain Ackermann steering geometry and its importance
5. Simulate vehicle trajectories using different models
6. Apply coordinate transformations for sensor fusion

---

## Setup

Import required libraries for vehicle modeling and visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyArrow, Circle
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Set random seed for reproducibility
np.random.seed(42)

# Plotting configuration
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

## 1. Kinematic Bicycle Model

The **kinematic bicycle model** is a simplified 2D model that approximates a four-wheeled vehicle as a bicycle. It's widely used in autonomous driving for **path planning and control** due to its simplicity and effectiveness at low-to-medium speeds.

### Why "Bicycle" Model?

A car has 4 wheels, but we can simplify:
- **Front two wheels** → Single front wheel (steered)
- **Rear two wheels** → Single rear wheel (fixed, no steering)

This simplification is valid because:
- At low speeds, tire slip is negligible
- The two wheels on each axle move together
- We only care about the vehicle's center of gravity motion

### Model Assumptions

✅ No tire slip (wheels roll without sliding)  
✅ Small steering angles (valid for normal driving)  
✅ Constant velocity or slowly varying acceleration  
✅ Flat, level road  
❌ Not valid at high speeds or aggressive maneuvers (use dynamic model)

---

### Vehicle Geometry

```
                    Front Axle
                        ↓
                    ╭───●───╮     ← Front wheel (steered)
                    │   ↑   │
                    │   │   │
            L_f     │   │   │     L = wheelbase
                    │   ● CG│     (distance between axles)
                    │   │   │
                    │   │   │
            L_r     │   │   │
                    ╰───●───╯     ← Rear wheel (fixed)
                        ↑
                    Rear Axle
```

**Parameters**:
- **L**: Wheelbase (distance between front and rear axles) ≈ 2.5-3.0m for sedans
- **L_f**: Distance from CG to front axle
- **L_r**: Distance from CG to rear axle (L = L_f + L_r)
- **δ (delta)**: Steering angle at front wheel

---

### State Variables

The vehicle state is represented by:

| Variable | Symbol | Description | Units |
|----------|--------|-------------|-------|
| Position (x) | x | Global x-coordinate of rear axle | m |
| Position (y) | y | Global y-coordinate of rear axle | m |
| Heading | ψ (psi) | Orientation angle (yaw) | rad |
| Velocity | v | Speed at rear axle | m/s |

**State vector**: **s** = [x, y, ψ, v]ᵀ

---

### Kinematic Equations (Rear-Wheel Reference)

The motion is governed by:

$$\begin{align}
\dot{x} &= v \cos(\psi) \\
\dot{y} &= v \sin(\psi) \\
\dot{\psi} &= \frac{v}{L} \tan(\delta) \\
\dot{v} &= a
\end{align}$$

Where:
- **v**: Velocity of the rear axle
- **δ**: Steering angle (control input)
- **a**: Acceleration (control input)
- **L**: Wheelbase

**Control inputs**: **u** = [δ, a]ᵀ

---

### Derivation of Heading Rate

The key equation is the **heading rate** (yaw rate):

$$\dot{\psi} = \frac{v}{L} \tan(\delta)$$

**Geometric derivation**:

1. The vehicle follows a **circular arc** with radius **R**
2. From geometry: **tan(δ) = L / R**
3. Angular velocity: **ω = v / R**
4. Substitute: **ω = v / (L / tan(δ)) = (v / L) tan(δ)**

For **small steering angles** (δ < 10°), we can use the approximation:
$$\tan(\delta) \approx \delta \quad \Rightarrow \quad \dot{\psi} \approx \frac{v \delta}{L}$$

---

### Alternative: Front-Wheel Reference Point

If we track the **front axle** instead of rear axle:

$$\begin{align}
\dot{x}_f &= v_f \cos(\psi + \delta) \\
\dot{y}_f &= v_f \sin(\psi + \delta) \\
\dot{\psi} &= \frac{v_f}{L} \sin(\delta)
\end{align}$$

Most implementations use **rear-wheel reference** for simpler equations.

---

### Practical Example

**Scenario**: Car traveling at **v = 10 m/s** with wheelbase **L = 2.5 m**, steering angle **δ = 5° = 0.087 rad**

Calculate the turning radius and heading rate:

1. **Turning radius**: R = L / tan(δ) = 2.5 / tan(0.087) ≈ **28.7 m**
2. **Heading rate**: ψ̇ = v tan(δ) / L = 10 × tan(0.087) / 2.5 ≈ **0.35 rad/s** (20°/s)

This means the vehicle completes a full circle in:
- **Time**: T = 2πR / v = 2π × 28.7 / 10 ≈ **18 seconds**

In [ ]:
# Kinematic Bicycle Model Implementation

class KinematicBicycleModel:
    """
    Implements the kinematic bicycle model for vehicle motion.
    
    State: [x, y, psi, v]
    Control: [delta, a]
    """
    
    def __init__(self, wheelbase=2.5, max_steer=np.deg2rad(30), dt=0.1):
        """
        Args:
            wheelbase: Distance between front and rear axles (m)
            max_steer: Maximum steering angle (rad)
            dt: Time step for simulation (s)
        """
        self.L = wheelbase
        self.max_steer = max_steer
        self.dt = dt
        
    def update(self, state, control):
        """
        Update state using kinematic bicycle model equations.
        
        Args:
            state: [x, y, psi, v] - current state
            control: [delta, a] - steering angle (rad), acceleration (m/s²)
            
        Returns:
            new_state: [x, y, psi, v] - updated state
        """
        x, y, psi, v = state
        delta, a = control
        
        # Clamp steering angle
        delta = np.clip(delta, -self.max_steer, self.max_steer)
        
        # Kinematic bicycle model equations
        dx = v * np.cos(psi)
        dy = v * np.sin(psi)
        dpsi = (v / self.L) * np.tan(delta)
        dv = a
        
        # Euler integration
        new_x = x + dx * self.dt
        new_y = y + dy * self.dt
        new_psi = psi + dpsi * self.dt
        new_v = v + dv * self.dt
        
        # Keep velocity non-negative
        new_v = max(0, new_v)
        
        return np.array([new_x, new_y, new_psi, new_v])
    
    def simulate_trajectory(self, initial_state, control_sequence):
        """
        Simulate vehicle trajectory given control sequence.
        
        Args:
            initial_state: [x, y, psi, v] - starting state
            control_sequence: List of [delta, a] controls
            
        Returns:
            trajectory: Array of states [N x 4]
        """
        trajectory = [initial_state]
        state = initial_state
        
        for control in control_sequence:
            state = self.update(state, control)
            trajectory.append(state.copy())
            
        return np.array(trajectory)


# Example: Simulate different maneuvers
model = KinematicBicycleModel(wheelbase=2.5, dt=0.1)

# Initial state: x=0, y=0, heading=0, velocity=10 m/s
initial_state = np.array([0.0, 0.0, 0.0, 10.0])

# Maneuver 1: Straight line (no steering)
straight_controls = [[0.0, 0.0] for _ in range(50)]
traj_straight = model.simulate_trajectory(initial_state, straight_controls)

# Maneuver 2: Left turn (constant steering angle)
left_turn_controls = [[np.deg2rad(5), 0.0] for _ in range(100)]
traj_left = model.simulate_trajectory(initial_state, left_turn_controls)

# Maneuver 3: Lane change (S-curve)
lane_change_controls = []
for i in range(30):
    lane_change_controls.append([np.deg2rad(8), 0.0])  # Steer left
for i in range(30):
    lane_change_controls.append([np.deg2rad(-8), 0.0])  # Steer right
for i in range(20):
    lane_change_controls.append([0.0, 0.0])  # Straighten
traj_lane_change = model.simulate_trajectory(initial_state, lane_change_controls)

# Maneuver 4: Circle (constant steering, deceleration)
circle_controls = [[np.deg2rad(10), -0.2] for _ in range(150)]
traj_circle = model.simulate_trajectory(initial_state, circle_controls)

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 14))

maneuvers = [
    (traj_straight, "Straight Line", "No steering"),
    (traj_left, "Left Turn", "5° constant steering"),
    (traj_lane_change, "Lane Change", "S-curve maneuver"),
    (traj_circle, "Circular Path", "10° steering + deceleration")
]

for ax, (traj, title, subtitle) in zip(axes.flat, maneuvers):
    # Plot trajectory
    ax.plot(traj[:, 0], traj[:, 1], 'b-', linewidth=2, label='Trajectory')
    ax.plot(traj[0, 0], traj[0, 1], 'go', markersize=12, label='Start')
    ax.plot(traj[-1, 0], traj[-1, 1], 'ro', markersize=12, label='End')
    
    # Draw vehicle at several positions
    for i in [0, len(traj)//2, len(traj)-1]:
        x, y, psi, v = traj[i]
        # Vehicle rectangle
        vehicle_length = 4.5
        vehicle_width = 2.0
        
        # Vehicle corners in local frame
        corners = np.array([
            [-vehicle_length/2, -vehicle_width/2],
            [vehicle_length/2, -vehicle_width/2],
            [vehicle_length/2, vehicle_width/2],
            [-vehicle_length/2, vehicle_width/2],
            [-vehicle_length/2, -vehicle_width/2]
        ])
        
        # Rotation matrix
        R = np.array([[np.cos(psi), -np.sin(psi)],
                      [np.sin(psi), np.cos(psi)]])
        
        # Transform to global frame
        corners_global = (R @ corners.T).T + np.array([x, y])
        
        alpha = 0.3 if i == len(traj)//2 else 0.6
        ax.plot(corners_global[:, 0], corners_global[:, 1], 'k-', linewidth=1.5, alpha=alpha)
        
        # Heading arrow
        arrow_length = 3
        dx = arrow_length * np.cos(psi)
        dy = arrow_length * np.sin(psi)
        ax.arrow(x, y, dx, dy, head_width=1, head_length=1, fc='red', ec='red', alpha=alpha)
    
    ax.set_xlabel('X Position (m)', fontweight='bold')
    ax.set_ylabel('Y Position (m)', fontweight='bold')
    ax.set_title(f'{title}\n{subtitle}', fontweight='bold', fontsize=12)
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    ax.axis('equal')

plt.tight_layout()
plt.suptitle('Kinematic Bicycle Model: Vehicle Maneuvers', 
             fontsize=16, fontweight='bold', y=1.00)
plt.show()

# Print statistics
print("=" * 70)
print("TRAJECTORY STATISTICS")
print("=" * 70)
for name, (traj, title, _) in zip(['Straight', 'Left Turn', 'Lane Change', 'Circle'], maneuvers):
    distance = np.sum(np.sqrt(np.diff(traj[:, 0])**2 + np.diff(traj[:, 1])**2))
    final_heading = np.rad2deg(traj[-1, 2])
    final_velocity = traj[-1, 3]
    
    print(f"\n{title}:")
    print(f"  Distance traveled: {distance:.2f} m")
    print(f"  Final heading: {final_heading:.2f}°")
    print(f"  Final velocity: {final_velocity:.2f} m/s")
    print(f"  Final position: ({traj[-1, 0]:.2f}, {traj[-1, 1]:.2f})")
print("=" * 70)

## Exercises

### Exercise 1: Kinematic Bicycle Model Derivation

A vehicle with wheelbase **L = 3.0 m** is traveling at **v = 15 m/s** with steering angle **δ = 8°**.

**a)** Calculate the turning radius R.

**b)** Calculate the heading rate (yaw rate) ψ̇ in rad/s and deg/s.

**c)** How long does it take to complete a full 360° turn?

**d)** If the vehicle maintains this steering angle for 5 seconds, what is the total heading change?

---

### Exercise 2: Coordinate Transformation

A LiDAR sensor is mounted on a vehicle with the following configuration:
- **Vehicle position (global)**: (50, 30) m
- **Vehicle heading**: 60°
- **LiDAR position (vehicle frame)**: (1.5, 0) m (1.5m forward from center)
- **LiDAR detects object at**: (10, 2) m in LiDAR frame

**a)** Create the homogeneous transformation matrix from vehicle frame to global frame.

**b)** Calculate the LiDAR's global position.

**c)** Calculate the object's global position.

**d)** What is the distance from the vehicle center to the object?

---

### Exercise 3: Ackermann Steering

A sedan has the following specifications:
- **Wheelbase**: L = 2.8 m
- **Track width**: t = 1.5 m
- **Maximum steering angle**: δ_max = 32°

**a)** For an average steering angle of δ = 20°, calculate the turning radius R.

**b)** Calculate the inner and outer wheel steering angles (δ_i and δ_o).

**c)** What is the minimum turning radius (using maximum steering angle)?

**d)** What is the approximate curb-to-curb turning diameter?

**e)** Can this vehicle make a U-turn on a street with 12m width?

---

### Exercise 4: Trajectory Prediction

Using the kinematic bicycle model, predict the vehicle's position after 3 seconds given:
- **Initial state**: x=0, y=0, ψ=0, v=12 m/s
- **Wheelbase**: L = 2.5 m
- **Constant inputs**: δ = 5°, a = 0 m/s² (no acceleration)
- **Time step**: dt = 0.1 s

**a)** Implement the kinematic equations to simulate the trajectory.

**b)** What is the final position (x, y)?

**c)** What is the final heading ψ (in degrees)?

**d)** Plot the trajectory.

---

### Exercise 5: Sensor Frame Fusion

Three sensors detect the same obstacle:
- **Camera** (vehicle frame pos: [2, 0]): Detects obstacle at [8, -1] in camera frame
- **LiDAR** (vehicle frame pos: [1, 0.5]): Detects obstacle at [7.2, -1.6] in LiDAR frame
- **Radar** (vehicle frame pos: [2, -0.5]): Detects obstacle at [8.1, -0.4] in radar frame

Vehicle is at global position [0, 0] with heading 0°.

**a)** Transform all three detections to the global frame.

**b)** Calculate the mean obstacle position (sensor fusion).

**c)** Calculate the standard deviation of the measurements (uncertainty).

**d)** Which sensor is most accurate (closest to mean)?

In [ ]:
# Exercise Solutions

print("=" * 70)
print("EXERCISE 1: Kinematic Bicycle Model")
print("=" * 70)

L = 3.0  # m
v = 15.0  # m/s
delta = np.deg2rad(8)  # rad

# a) Turning radius
R = L / np.tan(delta)
print(f"\na) Turning radius:")
print(f"   R = L / tan(δ) = {L} / tan({np.rad2deg(delta)}°)")
print(f"   R = {R:.2f} m")

# b) Heading rate
psi_dot = (v / L) * np.tan(delta)
psi_dot_deg = np.rad2deg(psi_dot)
print(f"\nb) Heading rate (yaw rate):")
print(f"   ψ̇ = (v/L) × tan(δ) = ({v}/{L}) × tan({np.rad2deg(delta)}°)")
print(f"   ψ̇ = {psi_dot:.4f} rad/s = {psi_dot_deg:.2f} deg/s")

# c) Time for 360° turn
time_full_circle = 2 * np.pi / psi_dot
print(f"\nc) Time for 360° turn:")
print(f"   T = 2π / ψ̇ = 2π / {psi_dot:.4f}")
print(f"   T = {time_full_circle:.2f} seconds")

# d) Heading change after 5 seconds
t = 5.0
heading_change = psi_dot * t
heading_change_deg = np.rad2deg(heading_change)
print(f"\nd) Heading change after {t} seconds:")
print(f"   Δψ = ψ̇ × t = {psi_dot:.4f} × {t}")
print(f"   Δψ = {heading_change:.4f} rad = {heading_change_deg:.2f}°")

print("\n" + "=" * 70)
print("EXERCISE 2: Coordinate Transformation")
print("=" * 70)

# Given data
vehicle_pos = np.array([50, 30])
vehicle_heading = np.deg2rad(60)
lidar_pos_V = np.array([1.5, 0])
object_pos_L = np.array([10, 2])

# a) Transformation matrix
T_V_to_G = CoordinateTransform.homogeneous_transform_2d(
    vehicle_pos[0], vehicle_pos[1], vehicle_heading)
print(f"\na) Vehicle-to-Global transformation matrix:")
print(T_V_to_G)

# b) LiDAR global position
lidar_pos_G = CoordinateTransform.transform_point_2d(lidar_pos_V, T_V_to_G)
print(f"\nb) LiDAR global position:")
print(f"   LiDAR (vehicle frame): {lidar_pos_V}")
print(f"   LiDAR (global frame): ({lidar_pos_G[0]:.2f}, {lidar_pos_G[1]:.2f})")

# c) Object global position
# First: LiDAR frame to vehicle frame
object_pos_V = lidar_pos_V + object_pos_L  # Simplified (assuming aligned)
# Then: Vehicle frame to global frame
object_pos_G = CoordinateTransform.transform_point_2d(object_pos_V, T_V_to_G)
print(f"\nc) Object global position:")
print(f"   Object (LiDAR frame): {object_pos_L}")
print(f"   Object (vehicle frame): {object_pos_V}")
print(f"   Object (global frame): ({object_pos_G[0]:.2f}, {object_pos_G[1]:.2f})")

# d) Distance from vehicle to object
distance = np.linalg.norm(object_pos_G - vehicle_pos)
print(f"\nd) Distance from vehicle to object:")
print(f"   d = ||object - vehicle|| = {distance:.2f} m")

print("\n" + "=" * 70)
print("EXERCISE 3: Ackermann Steering")
print("=" * 70)

L = 2.8  # m
t = 1.5  # m
delta_avg = np.deg2rad(20)  # rad
delta_max = np.deg2rad(32)  # rad

# a) Turning radius at 20°
R = L / np.tan(delta_avg)
print(f"\na) Turning radius at δ = 20°:")
print(f"   R = L / tan(δ) = {L} / tan({np.rad2deg(delta_avg)}°)")
print(f"   R = {R:.2f} m")

# b) Inner and outer wheel angles
# Using approximation: δ_i ≈ δ + (t/L)×δ, δ_o ≈ δ - (t/L)×δ
delta_diff = (t / L) * delta_avg
delta_i = delta_avg + delta_diff
delta_o = delta_avg - delta_diff
print(f"\nb) Wheel steering angles:")
print(f"   Δδ = (t/L) × δ = ({t}/{L}) × {np.rad2deg(delta_avg)}° = {np.rad2deg(delta_diff):.2f}°")
print(f"   δ_inner = δ + Δδ = {np.rad2deg(delta_avg):.1f}° + {np.rad2deg(delta_diff):.2f}° = {np.rad2deg(delta_i):.2f}°")
print(f"   δ_outer = δ - Δδ = {np.rad2deg(delta_avg):.1f}° - {np.rad2deg(delta_diff):.2f}° = {np.rad2deg(delta_o):.2f}°")

# c) Minimum turning radius
R_min = L / np.tan(delta_max)
print(f"\nc) Minimum turning radius (δ_max = {np.rad2deg(delta_max)}°):")
print(f"   R_min = L / tan(δ_max) = {L} / tan({np.rad2deg(delta_max)}°)")
print(f"   R_min = {R_min:.2f} m")

# d) Curb-to-curb diameter
overhang = 1.0  # typical overhang
diameter = 2 * (R_min + overhang)
print(f"\nd) Curb-to-curb turning diameter:")
print(f"   Diameter ≈ 2 × (R_min + overhang)")
print(f"   Diameter ≈ 2 × ({R_min:.2f} + {overhang}) = {diameter:.2f} m")

# e) Can it make U-turn?
street_width = 12.0
can_uturn = diameter <= street_width
print(f"\ne) U-turn on {street_width}m street:")
print(f"   Required: {diameter:.2f} m")
print(f"   Available: {street_width} m")
print(f"   Can make U-turn: {'✓ YES' if can_uturn else '✗ NO'}")

print("\n" + "=" * 70)
print("EXERCISE 4: Trajectory Prediction")
print("=" * 70)

# Initial conditions
initial_state = np.array([0.0, 0.0, 0.0, 12.0])  # x, y, psi, v
L = 2.5
delta = np.deg2rad(5)
a = 0.0
dt = 0.1
duration = 3.0

# Simulate
model = KinematicBicycleModel(wheelbase=L, dt=dt)
n_steps = int(duration / dt)
controls = [[delta, a] for _ in range(n_steps)]
trajectory = model.simulate_trajectory(initial_state, controls)

final_state = trajectory[-1]
print(f"\na) Simulation with dt={dt}s for {duration}s ({n_steps} steps)")
print(f"   Initial state: x={initial_state[0]}, y={initial_state[1]}, ψ={np.rad2deg(initial_state[2])}°, v={initial_state[3]} m/s")
print(f"   Control inputs: δ={np.rad2deg(delta)}°, a={a} m/s²")

print(f"\nb) Final position:")
print(f"   x = {final_state[0]:.2f} m")
print(f"   y = {final_state[1]:.2f} m")

print(f"\nc) Final heading:")
print(f"   ψ = {np.rad2deg(final_state[2]):.2f}°")

print(f"\nd) Trajectory plot:")
# Simple plot
fig, ax = plt.subplots(figsize=(10, 8))
ax.plot(trajectory[:, 0], trajectory[:, 1], 'b-', linewidth=2, label='Trajectory')
ax.plot(trajectory[0, 0], trajectory[0, 1], 'go', markersize=12, label='Start')
ax.plot(trajectory[-1, 0], trajectory[-1, 1], 'ro', markersize=12, label='End (t=3s)')
ax.set_xlabel('X Position (m)', fontweight='bold')
ax.set_ylabel('Y Position (m)', fontweight='bold')
ax.set_title('Predicted Trajectory (Kinematic Bicycle Model)', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axis('equal')
plt.show()

print("\n" + "=" * 70)
print("EXERCISE 5: Sensor Frame Fusion")
print("=" * 70)

# Sensor configurations (position in vehicle frame)
sensors_config = {
    'Camera': {'pos_V': np.array([2.0, 0.0]), 'detection': np.array([8.0, -1.0])},
    'LiDAR': {'pos_V': np.array([1.0, 0.5]), 'detection': np.array([7.2, -1.6])},
    'Radar': {'pos_V': np.array([2.0, -0.5]), 'detection': np.array([8.1, -0.4])}
}

vehicle_pos = np.array([0.0, 0.0])
vehicle_heading = 0.0

T_V_to_G = CoordinateTransform.homogeneous_transform_2d(
    vehicle_pos[0], vehicle_pos[1], vehicle_heading)

# a) Transform all detections
detections_global = {}
print("\na) Transform detections to global frame:")
for sensor_name, config in sensors_config.items():
    # Sensor detection in vehicle frame = sensor position + detection
    detection_V = config['pos_V'] + config['detection']
    # Vehicle frame to global frame
    detection_G = CoordinateTransform.transform_point_2d(detection_V, T_V_to_G)
    detections_global[sensor_name] = detection_G
    print(f"   {sensor_name}: ({detection_G[0]:.2f}, {detection_G[1]:.2f})")

# b) Mean position (fusion)
all_detections = np.array(list(detections_global.values()))
mean_pos = np.mean(all_detections, axis=0)
print(f"\nb) Fused obstacle position (mean):")
print(f"   Mean: ({mean_pos[0]:.2f}, {mean_pos[1]:.2f})")

# c) Standard deviation (uncertainty)
std_dev = np.std(all_detections, axis=0)
total_std = np.linalg.norm(std_dev)
print(f"\nc) Measurement uncertainty:")
print(f"   Std dev (x, y): ({std_dev[0]:.3f}, {std_dev[1]:.3f})")
print(f"   Total std dev: {total_std:.3f} m")

# d) Most accurate sensor
print(f"\nd) Sensor accuracy (distance from mean):")
errors = {}
for sensor_name, detection in detections_global.items():
    error = np.linalg.norm(detection - mean_pos)
    errors[sensor_name] = error
    print(f"   {sensor_name}: {error:.3f} m")

most_accurate = min(errors, key=errors.get)
print(f"\n   Most accurate: {most_accurate} (error = {errors[most_accurate]:.3f} m)")

print("=" * 70)

---

## References

### Textbooks

- **Rajamani, R.** (2012). *Vehicle Dynamics and Control* (2nd ed.). Springer.
  - Comprehensive treatment of vehicle modeling, including bicycle models and tire dynamics

- **Jazar, R. N.** (2017). *Vehicle Dynamics: Theory and Application* (3rd ed.). Springer.
  - Detailed coverage of coordinate frames, kinematics, and Ackermann steering

- **Paden, B., et al.** (2016). "A Survey of Motion Planning and Control Techniques for Self-Driving Urban Vehicles". *IEEE Transactions on Intelligent Vehicles*, 1(1), 33-55.
  - Survey of vehicle models used in autonomous driving

### Research Papers

- **Kong, J., et al.** (2015). "Kinematic and Dynamic Vehicle Models for Autonomous Driving Control Design". *IEEE Intelligent Vehicles Symposium*.
  - Comparison of kinematic vs. dynamic models for control

- **Snider, J. M.** (2009). "Automatic Steering Methods for Autonomous Automobile Path Tracking". *Robotics Institute, Carnegie Mellon University*, CMU-RI-TR-09-08.
  - Path tracking using bicycle model

- **Polack, P., et al.** (2017). "The Kinematic Bicycle Model: A Consistent Model for Planning Feasible Trajectories for Autonomous Vehicles?". *IEEE Intelligent Vehicles Symposium*.
  - Analysis of bicycle model validity

### Coordinate Transformations

- **Craig, J. J.** (2017). *Introduction to Robotics: Mechanics and Control* (4th ed.). Pearson.
  - Excellent coverage of rotation matrices, Euler angles, and homogeneous transformations

- **Sola, J.** (2017). "Quaternion kinematics for the error-state Kalman filter". *arXiv:1711.02508*.
  - Advanced rotation representations for sensor fusion

### Ackermann Steering

- **Ackermann, R.** (1818). British Patent 4212: *A New Method of Directing Wheel Carriages*.
  - Original patent for Ackermann steering geometry

- **Dixon, J. C.** (1996). *Tires, Suspension and Handling* (2nd ed.). SAE International.
  - Practical aspects of steering geometry and vehicle handling

### Online Resources

- **MATLAB Vehicle Dynamics Blockset**: [https://www.mathworks.com/products/vehicle-dynamics.html](https://www.mathworks.com/products/vehicle-dynamics.html)
  - Tutorials and examples for vehicle modeling

- **Baidu Apollo Planning Module**: [https://github.com/ApolloAuto/apollo/tree/master/modules/planning](https://github.com/ApolloAuto/apollo/tree/master/modules/planning)
  - Open-source implementation using bicycle model

- **Coursera: Self-Driving Cars Specialization** (University of Toronto)
  - Module 1 covers vehicle kinematics and dynamics

### Software Libraries

- **PyDy** (Python Dynamics): [https://www.pydy.org/](https://www.pydy.org/)
  - Symbolic dynamics modeling in Python

- **SciPy Spatial Transformations**: [https://docs.scipy.org/doc/scipy/reference/spatial.transform.html](https://docs.scipy.org/doc/scipy/reference/spatial.transform.html)
  - Rotation and transformation utilities

---

## Summary

In this notebook, you learned:

1. **Kinematic Bicycle Model**: Simplified 2D vehicle model for path planning and control
   - State: [x, y, ψ, v]
   - Control: [δ, a]
   - Equations: ẋ = v cos(ψ), ẏ = v sin(ψ), ψ̇ = (v/L) tan(δ)

2. **Coordinate Transformations**: Converting between global, vehicle, and sensor frames
   - 2D: Rotation matrices and homogeneous transformations
   - 3D: Euler angles (roll, pitch, yaw) and rotation matrices
   - Applications: Sensor fusion, localization

3. **Ackermann Steering**: Geometric arrangement for proper turning
   - Inner wheel steers more than outer wheel
   - Prevents tire scrubbing
   - Determines minimum turning radius

These concepts form the foundation for **vehicle motion planning**, **control**, and **sensor fusion** in autonomous driving systems.

---

**Next Steps**: Proceed to [Week 3: Control Theory & Actuation](week03_control_theory_actuation.ipynb) to learn how to design controllers (PID, MPC) that use these models.